In [16]:
## Import relevant packages
from LESO import System
from LESO import PhotoVoltaic, Wind, Lithium, FastCharger, Consumer, Grid, FinalBalance

In [17]:
#%% Define system and components
modelname = "VREEVHUB_Utrecht_plain"
lat, lon = 52.09, 5.15 # Utrecht
target_dir = "G:\\My Drive\\0 Thesis\\LESO results\\examples"

# initiate System component
system = System(latitude=lat, longitude=lon, model_name=modelname)

# initiate and define components
pv_s = PhotoVoltaic("PV Full south", dof=True)
pv_w = PhotoVoltaic("PV West", azimuth=90, dof=True, capex=0.55) # slightly cheaper (lower land use, straight-forward construction)
pv_e = PhotoVoltaic("PV East", azimuth=-90, dof=True, capex=0.55) # slightly cheaper (lower land use, straight-forward construction)
wind = Wind("Onshore turbine", dof=True)
bat = Lithium("Li-ion EES", dof=True)
charger = FastCharger("DC quickcharger", dof=False)
petrolstation = Consumer("Petrolstation E. demand")
grid = Grid("Grid connection", installed=150e3)
final = FinalBalance(name="curtailment_underload")

In [18]:
#%% add the components to the system
component_list = [pv_s, pv_w, pv_e, wind, bat, charger, petrolstation, final, grid]
system.add_components(component_list)

In [19]:
# Run the optimization

## PRINT
if False:
    system.pyomo_print()
    
## SOLVE
if True:
    system.optimize(
            objective='osc',        # overnight system cost
            time=None,              # resorts to default; year 8760h
            store=False,             # write-out to json
            solver='gurobi',        # default solver
            nonconvex=False,        # solver option (warning will show if needed)
            solve=True,             # solve or just create model
            tee=False,              # mute solver outputs
    )




Calculating time series for 9 components...


C:\Users\Sethv\#Universiteit Twente\GIT\.venvLESO\lib\site-packages\pvlib\solarposition.py:368: FutureWarning: casting datetime64[ns] values to int64 with .astype(...) is deprecated and will raise in a future version. Use .view(...) instead.
  unixtime = np.array(time.astype(np.int64)/10**9)


c:\users\sethv\#universiteit twente\git\leso\LESO\data\consumption.csv
---> Note: lithium2 does not have 'calculate_time_serie' function
---> Note: grid2 does not have 'calculate_time_serie' function
---> Note: FinalBalance does not have 'calculate_time_serie' function
pv4 size                 =  0.0 kW(h)
pv5 size                 =  352.8 kW(h)
pv6 size                 =  1170.1 kW(h)
wind2 size                 =  464.4 kW(h)
lithium2 size                 =  2111.2 kW(h)

Total system cost (objective)=  -338.6 k€
proceeding to hacky method of splitting power to pos/neg


In [26]:
# access the results :: use the AttrDict at system.results to get to the data of interest.

for key in system.results.components:
    component = system.results.components[key]
    if component.settings.get("installed", None):
        print(f"{component.name} installed capacity: {component.settings.installed/1e3} kW(h)")


PV Full south installed capacity: 9.927574677308414e-09 kW(h)
PV West installed capacity: 352.8411767548082 kW(h)
PV East installed capacity: 1170.1294517225058 kW(h)
Onshore turbine installed capacity: 464.4043772897149 kW(h)
DC quickcharger installed capacity: 0.004 kW(h)
Li-ion EES installed capacity: 2111.2327957457487 kW(h)
Grid connection installed capacity: 150.0 kW(h)
